In [ ]:
# Configuration
OPENAI_API_KEY = 'your_key_here'
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
MAX_FAILURES = 10  # Limit for demo

In [ ]:
# Load models
from src.inference import load_models, similarity_inference, supervised_inference
from src.data_loader import load_csvs
from src.preprocessor import create_chunks
from src.data_loader import load_pdfs

parts_vectorstore, pdf_vectorstore, clf, llm = load_models()
non_generic_parts, downtime_history = load_csvs()
pdf_texts = load_pdfs()
pdf_chunks = {name: create_chunks([text]) for name, text in pdf_texts.items()}

parts_list = non_generic_parts.apply(lambda row: f"{row.get('CAT Part', '')}: {row.get('SAP Material Description', '')}", axis=1).tolist()
failure_list = downtime_history.apply(lambda row: f"{row.get('Order', '')}: {row.get('Notification', '')}: {row.get('Order Type', '')}: {row.get('Order Long Text Description', '')}: {row.get('Notification Long Text Description', '')}: {row.get('Sort Field', '')}: {row.get('Total Costs', '')}: {row.get('Total Work Hours ', '')}", axis=1).tolist()

In [ ]:
# Similarity Inference
results = similarity_inference(failure_list[:MAX_FAILURES], parts_list)
print('Similarity Results:', results[:3])

In [ ]:
# Supervised Inference
results_sup = supervised_inference(failure_list[:MAX_FAILURES], parts_list, pdf_chunks)
print('Supervised Results:', results_sup[:3])